# Residual Networks


## 1 - Packages

In [ ]:
import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow

np.random.seed(1)
tf.random.set_seed(2)

## 3 - Building a Residual Network

### 3.1 - The Identity Block: identity_block


In [12]:

def identity_block(X, f, filters, training=True, initializer=random_uniform):
    """
    Implementation of the identity block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to random uniform initializer
    
    Returns:
    X -- output of the identity block, tensor of shape (m, n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)
    
    ## Second component of main path 
    X = Conv2D(filters = F2, kernel_size= f, strides = (1, 1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X) 

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size= 1, strides=(1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training) 
    
    ## Final step
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X) 

    return X

### 3.2 - The Convolutional Block: convolutional_block


In [18]:

def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- to set up the initial weights of a layer. Equals to Glorot uniform initializer, 
                   also called Xavier uniform initializer.
    
    Returns:
    X -- output of the convolutional block, tensor of shape (m, n_H, n_W, n_C)
    """
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ## MAIN PATH ##
    
    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)
    
    ## Second component of main path 
    X = Conv2D(filters=F2, kernel_size=f, strides=(1,1), padding='same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training) 
    X = Activation('relu')(X)

    ## Third component of main path 
    X = Conv2D(filters=F3, kernel_size=1, strides=(1, 1), padding='valid', kernel_initializer = initializer(seed=0))(X) 
    X = BatchNormalization(axis=3)(X, training=training)
    
    ## SHORTCUT PATH ##
    X_shortcut = Conv2D(filters=F3, kernel_size=1, strides=(s, s), padding='valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut, training=training)
    
    # Final step
    X = Activation('relu')(X)
    
    return X

## 4 - Building  ResNet Model (50 layers)


In [22]:

def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input tensor
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    
    ## Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
 
    # the 3 `identity_block`
    X = identity_block(X, f=3, filters=[128, 128, 512])
    X = identity_block(X, f=3, filters=[128, 128, 512])
    X = identity_block(X, f=3, filters=[128, 128, 512])

    # Stage 4
    # add `convolutional_block`
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)
    
    # the 5 `identity_block`
    X = identity_block(X, f=3, filters=[256, 256, 1024])
    X = identity_block(X, f=3, filters=[256, 256, 1024])
    X = identity_block(X, f=3, filters=[256, 256, 1024])
    X = identity_block(X, f=3, filters=[256, 256, 1024])
    X = identity_block(X, f=3, filters=[256, 256, 1024])

    # Stage 5 
    # add `convolutional_block` 
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    
    # the 2 `identity_block` 
    X = identity_block(X, f=3, filters=[512, 512, 2048])
    X = identity_block(X, f=3, filters=[512, 512, 2048])

    # AVGPOOL
    X = AveragePooling2D(pool_size=(2, 2))(X)
    
    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [23]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 zero_padding2d_1 (ZeroPadding2  (None, 70, 70, 3)   0           ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_93 (Conv2D)             (None, 32, 32, 64)   9472        ['zero_padding2d_1[0][0]']       
                                                                                                  
 batch_normalization_93 (BatchN  (None, 32, 32, 64)  256         ['conv2d_93[0][0]']          

                                                                                                  
 conv2d_103 (Conv2D)            (None, 15, 15, 256)  16640       ['activation_88[0][0]']          
                                                                                                  
 batch_normalization_103 (Batch  (None, 15, 15, 256)  1024       ['conv2d_103[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 add_28 (Add)                   (None, 15, 15, 256)  0           ['batch_normalization_103[0][0]',
                                                                  'activation_86[0][0]']          
                                                                                                  
 activation_89 (Activation)     (None, 15, 15, 256)  0           ['add_28[0][0]']                 
          

 add_31 (Add)                   (None, 8, 8, 512)    0           ['batch_normalization_113[0][0]',
                                                                  'activation_95[0][0]']          
                                                                                                  
 activation_98 (Activation)     (None, 8, 8, 512)    0           ['add_31[0][0]']                 
                                                                                                  
 conv2d_114 (Conv2D)            (None, 8, 8, 128)    65664       ['activation_98[0][0]']          
                                                                                                  
 batch_normalization_114 (Batch  (None, 8, 8, 128)   512         ['conv2d_114[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activatio

                                                                                                  
 batch_normalization_124 (Batch  (None, 4, 4, 256)   1024        ['conv2d_124[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_108 (Activation)    (None, 4, 4, 256)    0           ['batch_normalization_124[0][0]']
                                                                                                  
 conv2d_125 (Conv2D)            (None, 4, 4, 256)    590080      ['activation_108[0][0]']         
                                                                                                  
 batch_normalization_125 (Batch  (None, 4, 4, 256)   1024        ['conv2d_125[0][0]']             
 Normalization)                                                                                   
          

 activation_118 (Activation)    (None, 4, 4, 256)    0           ['batch_normalization_134[0][0]']
                                                                                                  
 conv2d_135 (Conv2D)            (None, 4, 4, 1024)   263168      ['activation_118[0][0]']         
                                                                                                  
 batch_normalization_135 (Batch  (None, 4, 4, 1024)  4096        ['conv2d_135[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 add_38 (Add)                   (None, 4, 4, 1024)   0           ['batch_normalization_135[0][0]',
                                                                  'activation_116[0][0]']         
                                                                                                  
 activatio

                                                                                                  
 add_41 (Add)                   (None, 2, 2, 2048)   0           ['batch_normalization_145[0][0]',
                                                                  'activation_125[0][0]']         
                                                                                                  
 activation_128 (Activation)    (None, 2, 2, 2048)   0           ['add_41[0][0]']                 
                                                                                                  
 average_pooling2d (AveragePool  (None, 1, 1, 2048)  0           ['activation_128[0][0]']         
 ing2D)                                                                                           
                                                                                                  
 flatten (Flatten)              (None, 2048)         0           ['average_pooling2d[0][0]']      
          

In [25]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [27]:
model.fit(X_train, Y_train, epochs = 10, batch_size = 32)

Epoch 1/10
34/34 [==============================] - 7s 42ms/step - loss: 1.8746 - accuracy: 0.4509
Epoch 2/10
34/34 [==============================] - 1s 27ms/step - loss: 0.8593 - accuracy: 0.6796
Epoch 3/10
34/34 [==============================] - 1s 27ms/step - loss: 0.2871 - accuracy: 0.9019
Epoch 4/10
34/34 [==============================] - 1s 27ms/step - loss: 0.2426 - accuracy: 0.9176
Epoch 5/10
34/34 [==============================] - 1s 27ms/step - loss: 0.1251 - accuracy: 0.9574
Epoch 6/10
34/34 [==============================] - 1s 27ms/step - loss: 0.1633 - accuracy: 0.9556
Epoch 7/10
34/34 [==============================] - 1s 27ms/step - loss: 0.3730 - accuracy: 0.8843
Epoch 8/10
34/34 [==============================] - 1s 26ms/step - loss: 0.2215 - accuracy: 0.9213
Epoch 9/10
34/34 [==============================] - 1s 27ms/step - loss: 0.0542 - accuracy: 0.9824
Epoch 10/10
34/34 [==============================] - 1s 27ms/step - loss: 0.0408 - accuracy: 0.9870


In [28]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

4/4 [==============================] - 1s 10ms/step - loss: 0.3306 - accuracy: 0.9167
Loss = 0.33057156205177307
Test Accuracy = 0.9166666865348816


## 5 - Test on Uploaded Image

In [ ]:
img_path = 'images/my_image.jpg'
img = image.load_img(img_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.0
print('Input image shape:', x.shape)
imshow(img)
prediction = model.predict(x)
print("Class prediction vector [p(0), p(1), p(2), p(3), p(4), p(5)] = ", prediction)
print("Class:", np.argmax(prediction))
